In [1]:
# Implementation of a Collaborative Filtering based Recommender System usi
#RBMs

In [2]:
# For data we will use the movie dataset acquired by Grouplens, this contains movies, users,
#movie ratings by these users(https://grouplens.org/datasets/movielens/)


In [4]:
!wget -O moviedataset.zip http://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip -o moviedataset.zip -d ./data

--2017-10-27 14:04:21--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org... 128.101.34.235
Connecting to files.grouplens.org|128.101.34.235|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: 'moviedataset.zip'

moviedataset.zip    100%[===================>]   5.64M   733KB/s    in 7.9s    

2017-10-27 14:04:29 (733 KB/s) - 'moviedataset.zip' saved [5917549/5917549]

Archive:  moviedataset.zip
checkdir:  cannot create extraction directory: /resources/data
           No such file or directory


In [5]:
!unzip -o moviedataset.zip -d ./data

Archive:  moviedataset.zip
   creating: ./data/ml-1m/
  inflating: ./data/ml-1m/movies.dat  
  inflating: ./data/ml-1m/ratings.dat  
  inflating: ./data/ml-1m/README     
  inflating: ./data/ml-1m/users.dat  


In [7]:
# #Now let's import the necessary libraries
# #To model and initialze the RBM
# 1. Tensorflow
# 2. Numpy
# To Manipulate the data
# 1. Numpy



In [8]:
#Tensorflow library. Used to implement machine learning models
import tensorflow as tf
#Numpy contains helpful functions for efficient mathematical calculations
import numpy as np
#Dataframe manipulation library
import pandas as pd
#Graph plotting library
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
# Now we can load in our data with Pandas. The .dat files use '::' 
# as a seperator. our data file also have no Headers

In [15]:
#Loading in the movies dataset
movies_df = pd.read_csv('/Users/shishir/Documents/botconnect Playground/Collaborative recommender using RBM's/data/ml-1m/movies.dat')

SyntaxError: invalid syntax (<ipython-input-15-05016128cbc4>, line 2)